In [1]:
from common import *

In [ ]:
from wpca import PCA as PCA_, WPCA, EMPCA
from sklearn.decomposition import (
    PCA, FastICA,FactorAnalysis as FA, KernelPCA, SparsePCA, MiniBatchSparsePCA as MBSparsePCA
)
from sklearn.preprocessing import QuantileTransformer as QT

In [ ]:
def the_complete_submatrix(df):
    '''Uses (the) `df` to construct `ff`, which is the "canonical" complete submatrix of `df` in long form'''
    filt = day_filter(df, 246)
    valid = filt.pivot('Day', 'Stock', 'Count')
    thedays = set(valid.T.sum().pipe(lambda x: x[x >= 1955]).index)
    _thestocks = filt[filt.Day.isin(thedays)].groupby('Day').Stock.agg(lambda x: set(x))
    thestocks = np.bitwise_and.reduce(_thestocks.values)
    return filt[filt.Day.isin(thedays) & filt.Stock.isin(thestocks)]
    
ff = the_complete_submatrix(df)

In [ ]:
def def_matrices(namespace, data, y=1, x=True):
    '''Params - namespace: e.g. `globals()`, data: `df` or sub-DataFrame of it
    Put convenience matrix variables into `namespace`'''
    #global table, market, Y, Ynnorm, W, ywmean, ywvar, Ywnorm
    if y:
        class var_map(O()):
            table = data
            market = data.groupby('Stock').Market.nth(0)
            Y = data.pivot('Day', 'Stock', 'y') * y
            Ya = Y.fillna(0)
            Ynnorm = (Y - Y.mean()) / Y.std(ddof=1)
            Ynnorma = Ynnorm.fillna(0)
            W = data.pivot('Day', 'Stock', 'w')
            Wa = W.fillna(0)
            ywmean = (Y * W).sum() / W.sum()
            ywvar = (W * (Y - ywmean) ** 2).sum() / (W.sum() - W.mean())
            Ywnorm = (Y - ywmean) / np.sqrt(ywvar)
            Ywnorma = Ywnorm.fillna(0)
        dict.update(namespace, **var_map)
    if x:
        pass

In [ ]:
def def_qtnormed(namespace, var_name, dist='uniform normal'):
    '''Params - namespace: e.g. `globals()`, var_name: `str` where matrix is stored in `namespace`, dist: 'uniform'|'normal'
    Put quantile transformed versions of matrix at `var_name` into `namespace`.
    The variable name suffixes are ~cnorm, ~crnorm, ~rnorm, ~'''
    Y = namespace[var_name]
    if 'uniform' in dist:
        suf = 'unif'
        trans = QT(output_distribution='uniform')
        namespace[var_name + 'c' + suf] = Ycnorm = Y * 0 + trans.fit_transform(Y)
        namespace[var_name + 'cr' + suf] = Ycrnorm = (Ycnorm.T * 0 + trans.fit_transform(Ycnorm.T)).T
        namespace[var_name + 'r' + suf] = Yrnorm = (Y.T * 0 + trans.fit_transform(Y.T)).T
        namespace[var_name + 'rc' + suf] = Yrcnorm = Yrnorm.T * 0 + trans.fit_transform(Yrnorm.T)
    if 'normal' in dist:
        suf = 'norm'
        trans = QT(output_distribution='normal')
        namespace[var_name + 'c' + suf] = Ycnorm = Y * 0 + trans.fit_transform(Y)
        namespace[var_name + 'cr' + suf] = Ycrnorm = (Ycnorm.T * 0 + trans.fit_transform(Ycnorm.T)).T
        namespace[var_name + 'r' + suf] = Yrnorm = (Y.T * 0 + trans.fit_transform(Y.T)).T
        namespace[var_name + 'rc' + suf] = Yrcnorm = Yrnorm.T * 0 + trans.fit_transform(Yrnorm.T)